##Importing the libraries


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

##Importing the dataset


In [2]:
credits = pd.read_csv("/content/drive/MyDrive/Movie-Recommendation-System/credits.csv")

In [3]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [4]:
meta = pd.read_csv("/content/drive/MyDrive/Movie-Recommendation-System/movies_metadata.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
meta.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


##Preprocessing the dataset

In [6]:
meta["release_date"] = pd.to_datetime(meta["release_date"], errors="coerce")
print(meta["release_date"])

0       1995-10-30
1       1995-12-15
2       1995-12-22
3       1995-12-22
4       1995-02-10
           ...    
45461          NaT
45462   2011-11-17
45463   2003-08-01
45464   1917-10-21
45465   2017-06-09
Name: release_date, Length: 45466, dtype: datetime64[ns]


In [7]:
meta['year'] = meta['release_date'].dt.year

In [8]:
meta.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995.0


In [9]:
meta["year"].value_counts().sort_index()

1874.0       1
1878.0       1
1883.0       1
1887.0       1
1888.0       2
          ... 
2015.0    1905
2016.0    1604
2017.0     532
2018.0       5
2020.0       1
Name: year, Length: 135, dtype: int64

In [10]:
##Since we have the data upto 2016 in MovieProcessor.ipynb dataset.csv
##we will grab data of 2017 only
new_meta = meta.loc[meta.year == 2017, ["genres","title","id","year"]]
new_meta

,genres,title,id,year
26560,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",Pirates of the Caribbean: Dead Men Tell No Tales,166426,2017.0
26561,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Justice League,141052,2017.0
26565,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Thor: Ragnarok,284053,2017.0
26566,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Guardians of the Galaxy Vol. 2,283995,2017.0
30536,"[{'id': 14, 'name': 'Fantasy'}, {'id': 28, 'na...",The King's Daughter,245842,2017.0
...,...,...,...,...
45398,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Thick Lashes of Lauri Mäntyvaara,468707,2017.0
45417,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",Cop and a Half: New Recruit,461297,2017.0
45437,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",In a Heartbeat,455661,2017.0
45453,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",Mom,404604,2017.0


In [11]:
new_meta["id"] = new_meta['id'].astype(int)   ###change the dtype of id ccolumn from object to int
new_meta["id"]

26560    166426
26561    141052
26565    284053
26566    283995
30536    245842
          ...  
45398    468707
45417    461297
45437    455661
45453    404604
45465    461257
Name: id, Length: 532, dtype: int64

In [12]:
data = pd.merge(new_meta, credits, on='id')

In [13]:
data

,genres,title,id,year,cast,crew
0,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",Pirates of the Caribbean: Dead Men Tell No Tales,166426,2017.0,"[{'cast_id': 1, 'character': 'Captain Jack Spa...","[{'credit_id': '52fe4c9cc3a36847f8236a65', 'de..."
1,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Justice League,141052,2017.0,"[{'cast_id': 2, 'character': 'Bruce Wayne / Ba...","[{'credit_id': '55ef66dbc3a3686f1700a52d', 'de..."
2,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Thor: Ragnarok,284053,2017.0,"[{'cast_id': 0, 'character': 'Thor Odinson', '...","[{'credit_id': '56a93fa4c3a36872db001e7a', 'de..."
3,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Guardians of the Galaxy Vol. 2,283995,2017.0,"[{'cast_id': 3, 'character': 'Peter Quill / St...","[{'credit_id': '59171547925141583c0315a6', 'de..."
4,"[{'id': 14, 'name': 'Fantasy'}, {'id': 28, 'na...",The King's Daughter,245842,2017.0,"[{'cast_id': 0, 'character': 'King Louis XIV',...","[{'credit_id': '5431de49c3a36825d300007e', 'de..."
...,...,...,...,...,...,...
526,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Thick Lashes of Lauri Mäntyvaara,468707,2017.0,"[{'cast_id': 0, 'character': 'Satu', 'credit_i...","[{'credit_id': '597e22f69251415d7801c74a', 'de..."
527,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",Cop and a Half: New Recruit,461297,2017.0,"[{'cast_id': 0, 'character': 'Detective Simmon...","[{'credit_id': '593ba0c29251410593009be3', 'de..."
528,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",In a Heartbeat,455661,2017.0,[],"[{'credit_id': '5981a15c92514151e0011b51', 'de..."
529,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",Mom,404604,2017.0,"[{'cast_id': 1, 'character': 'Devki Sabarwal',...","[{'credit_id': '58ee55bbc3a3683df500bd0f', 'de..."


In [14]:
import ast               
###ast.literal.eval safely evaluate an expression node or string
data['genres'] = data['genres'].map(lambda x: ast.literal_eval(x))
data['cast'] = data['cast'].map(lambda x: ast.literal_eval(x))
data['crew'] = data['crew'].map(lambda x: ast.literal_eval(x))

In [15]:
data.head()

,genres,title,id,year,cast,crew
0,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",Pirates of the Caribbean: Dead Men Tell No Tales,166426,2017.0,"[{'cast_id': 1, 'character': 'Captain Jack Spa...","[{'credit_id': '52fe4c9cc3a36847f8236a65', 'de..."
1,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Justice League,141052,2017.0,"[{'cast_id': 2, 'character': 'Bruce Wayne / Ba...","[{'credit_id': '55ef66dbc3a3686f1700a52d', 'de..."
2,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Thor: Ragnarok,284053,2017.0,"[{'cast_id': 0, 'character': 'Thor Odinson', '...","[{'credit_id': '56a93fa4c3a36872db001e7a', 'de..."
3,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Guardians of the Galaxy Vol. 2,283995,2017.0,"[{'cast_id': 3, 'character': 'Peter Quill / St...","[{'credit_id': '59171547925141583c0315a6', 'de..."
4,"[{'id': 14, 'name': 'Fantasy'}, {'id': 28, 'na...",The King's Daughter,245842,2017.0,"[{'cast_id': 0, 'character': 'King Louis XIV',...","[{'credit_id': '5431de49c3a36825d300007e', 'de..."


In [16]:
def make_genresList(x):
    gen = []
    st = " "
    for i in x:
        if i.get('name') == 'Science Fiction': 
            scifi = 'Sci-Fi'
            gen.append(scifi)
        else:
            gen.append(i.get('name'))
    if gen == []:
        return np.NaN
    else:
        return (st.join(gen))

In [17]:
data["genres_list"] = data["genres"].map(lambda x: make_genresList(x))
data

,genres,title,id,year,cast,crew,genres_list
0,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",Pirates of the Caribbean: Dead Men Tell No Tales,166426,2017.0,"[{'cast_id': 1, 'character': 'Captain Jack Spa...","[{'credit_id': '52fe4c9cc3a36847f8236a65', 'de...",Adventure Action Fantasy Comedy
1,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Justice League,141052,2017.0,"[{'cast_id': 2, 'character': 'Bruce Wayne / Ba...","[{'credit_id': '55ef66dbc3a3686f1700a52d', 'de...",Action Adventure Fantasy Sci-Fi
2,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Thor: Ragnarok,284053,2017.0,"[{'cast_id': 0, 'character': 'Thor Odinson', '...","[{'credit_id': '56a93fa4c3a36872db001e7a', 'de...",Action Adventure Fantasy Sci-Fi
3,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Guardians of the Galaxy Vol. 2,283995,2017.0,"[{'cast_id': 3, 'character': 'Peter Quill / St...","[{'credit_id': '59171547925141583c0315a6', 'de...",Action Adventure Comedy Sci-Fi
4,"[{'id': 14, 'name': 'Fantasy'}, {'id': 28, 'na...",The King's Daughter,245842,2017.0,"[{'cast_id': 0, 'character': 'King Louis XIV',...","[{'credit_id': '5431de49c3a36825d300007e', 'de...",Fantasy Action Adventure
...,...,...,...,...,...,...,...
526,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",Thick Lashes of Lauri Mäntyvaara,468707,2017.0,"[{'cast_id': 0, 'character': 'Satu', 'credit_i...","[{'credit_id': '597e22f69251415d7801c74a', 'de...",Romance Comedy
527,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",Cop and a Half: New Recruit,461297,2017.0,"[{'cast_id': 0, 'character': 'Detective Simmon...","[{'credit_id': '593ba0c29251410593009be3', 'de...",Crime Comedy Action Family
528,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",In a Heartbeat,455661,2017.0,[],"[{'credit_id': '5981a15c92514151e0011b51', 'de...",Family Animation Romance Comedy
529,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",Mom,404604,2017.0,"[{'cast_id': 1, 'character': 'Devki Sabarwal',...","[{'credit_id': '58ee55bbc3a3683df500bd0f', 'de...",Crime Drama Thriller


We have make columns for actor_1_name, actor_2_name, actor_3_name and director in MovieProcessor.ipynb datset.csv upto 2016 dataset. Now we will do same for the 2017 dataset.

In [18]:
def get_actor1(x):
    casts = []
    for i in x:
        casts.append(i.get('name'))
    if casts == []:
        return np.NaN
    else:
        return (casts[0])

In [19]:
data['actor_1_name'] = data['cast'].map(lambda x: get_actor1(x))

In [20]:
def get_actor2(x):
    casts = []
    for i in x:
        casts.append(i.get('name'))
    if casts == [] or len(casts)<=1:
        return np.NaN
    else:
        return (casts[1])

In [21]:
data['actor_2_name'] = data['cast'].map(lambda x: get_actor2(x))

In [22]:
def get_actor3(x):
    casts = []
    for i in x:
        casts.append(i.get('name'))
    if casts == [] or len(casts)<=2:
        return np.NaN
    else:
        return (casts[2])

In [23]:
data['actor_3_name'] = data['cast'].map(lambda x: get_actor3(x))

In [24]:
def get_directors(x):
    dt = []
    st = " "
    for i in x:
        if i.get('job') == 'Director':
            dt.append(i.get('name'))
    if dt == []:
        return np.NaN
    else:
        return (st.join(dt))

In [25]:
data['director_name'] = data['crew'].map(lambda x: get_directors(x))

In [26]:
##get columns
movie2017 = data.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres_list','title']]

In [27]:
movie2017.isna().sum()

director_name     4
actor_1_name     22
actor_2_name     55
actor_3_name     70
genres_list       7
title             0
dtype: int64

In [28]:
movie2017.shape

(531, 6)

In [29]:
movie2017 = movie2017.dropna(how="any")

In [30]:
movie2017.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres_list      0
title            0
dtype: int64

In [31]:
movie2017 = movie2017.rename(columns={'genres_list':'genres'})
movie2017 = movie2017.rename(columns={'title':'movie_title'})

In [32]:
movie2017['movie_title'] = movie2017['movie_title'].str.lower()
movie2017['comb'] = movie2017['actor_1_name'] + ' ' + movie2017['actor_2_name'] + ' '+ movie2017['actor_3_name'] + ' '+ movie2017['director_name'] +' ' + movie2017['genres']

In [33]:
movieupto2016 = pd.read_csv('dataset.csv')

In [34]:
movieupto2016.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...


In [35]:
movie2017.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Joachim Rønning Espen Sandberg,Johnny Depp,Javier Bardem,Geoffrey Rush,Adventure Action Fantasy Comedy,pirates of the caribbean: dead men tell no tales,Johnny Depp Javier Bardem Geoffrey Rush Joachi...
1,Zack Snyder,Ben Affleck,Henry Cavill,Gal Gadot,Action Adventure Fantasy Sci-Fi,justice league,Ben Affleck Henry Cavill Gal Gadot Zack Snyder...
2,Taika Waititi,Chris Hemsworth,Tom Hiddleston,Cate Blanchett,Action Adventure Fantasy Sci-Fi,thor: ragnarok,Chris Hemsworth Tom Hiddleston Cate Blanchett ...
3,James Gunn,Chris Pratt,Zoe Saldana,Dave Bautista,Action Adventure Comedy Sci-Fi,guardians of the galaxy vol. 2,Chris Pratt Zoe Saldana Dave Bautista James Gu...
4,Sean McNamara,Pierce Brosnan,William Hurt,Benjamin Walker,Fantasy Action Adventure,the king's daughter,Pierce Brosnan William Hurt Benjamin Walker Se...


In [36]:
movieupto2016.shape

(5043, 7)

In [37]:
new_dataset = movieupto2016.append(movie2017)

In [38]:
new_dataset.shape

(5501, 7)

In [39]:
new_dataset.drop_duplicates(subset ="movie_title", keep = 'last', inplace = True)   ###Remove duplicate rows on the basis of title

In [40]:
new_dataset.shape

(5375, 7)

In [41]:
new_dataset.to_csv("newdataset.csv",index=False)